##  Importer les bibliothèques 

In [1]:
import mlflow
import mlflow.sklearn
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import mlflow
from mlflow import MlflowClient
import warnings

## Chargement et prétraitement des données

In [2]:

test_df = pd.read_csv('../Data/test.csv')

print(test_df.columns)

warnings.filterwarnings('ignore')

# Remplir les valeurs manquantes pour 'Age' avec la médiane

test_df['Age'].fillna(test_df['Age'].median(), inplace=True)

# Remplir les valeurs manquantes pour 'Embarked' avec la valeur la plus fréquente
test_df['Embarked'].fillna(test_df['Embarked'].mode()[0], inplace=True)

# Remplir les valeurs manquantes pour 'Fare' dans le jeu de test avec la médiane
test_df['Fare'].fillna(test_df['Fare'].median(), inplace=True)

# Supprimer la colonne 'Cabin' car elle contient trop de valeurs manquantes
test_df.drop('Cabin', axis=1, inplace=True)

# Créer une nouvelle feature 'FamilySize' à partir de 'SibSp' et 'Parch'
test_df['FamilySize'] = test_df['SibSp'] + test_df['Parch'] + 1

# Créer une nouvelle feature 'IsAlone' à partir de 'FamilySize'
test_df['IsAlone'] = 1
test_df.loc[test_df['FamilySize'] > 1, 'IsAlone'] = 0

# Supprimer les colonnes inutiles
test_df = test_df.drop(['Name', 'Ticket', 'SibSp', 'Parch'], axis=1)

# Convertir la feature 'Sex' en numérique
test_df['Sex'] = test_df['Sex'].map({'male': 0, 'female': 1}).astype(int)

# Convertir la feature 'Embarked' en numérique en utilisant le one-hot encoding (à voir les tech)
test_df = pd.get_dummies(test_df, columns=['Embarked'])

Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')


## Séparation des données

In [3]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

NameError: name 'X' is not defined

## Recherche et affichage des modèles enregistrés avec Mlflow

In [4]:
client = MlflowClient()
model_name = "TitanicModel2"
filter_string = f"name='{model_name}'"
results = client.search_registered_models(filter_string=filter_string)
print(results)
print("-" * 80)
for res in results:
    for mv in res.latest_versions:
        run_id=mv.run_id
        print(f"name={mv.name}; run_id={mv.run_id}; version={mv.version}")






[<RegisteredModel: aliases={}, creation_timestamp=1719820693892, description=None, last_updated_timestamp=1720689049566, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1720689049566, current_stage='None', description=None, last_updated_timestamp=1720689049566, name='TitanicModel2', run_id='d9b8dc2e2b0842348dd6d11be4ee9c17', run_link=None, source='runs:/67f68e03d8784288a916d03901629301/model', status='READY', status_message=None, tags={}, user_id=None, version=21>], name='TitanicModel2', tags={}>]
--------------------------------------------------------------------------------
name=TitanicModel2; run_id=d9b8dc2e2b0842348dd6d11be4ee9c17; version=21


##  Charger la dernière version du modèle enregistré avec MLflow

In [5]:
registered_versions = mlflow_client.search_registered_models(filter_string=filter_string,
    order_by=(["last_updated_timestamp DESC"]))
print(registered_versions)
last_model_uri=registered_versions[0].latest_versions[-1].source    
print(f"last_model_uri : {last_model_uri}")    # Load the appropriate model identified above    

# Charger le modèle depuis MLflow
loaded_model = mlflow.sklearn.load_model(last_model_uri)

NameError: name 'mlflow_client' is not defined

## Appliquer le modèle pour faire des prédictions

In [6]:
# Appliquer le modèle pour faire des prédictions
predictions = loaded_model.predict(test_df)

# Générer le fichier submission.csv
submission_pred = pd.DataFrame({'PassengerId': test_df['PassengerId'], 'Survived': predictions})
submission_pred.to_csv('../Results/submission_pred.csv', index=False)

print("Les prédictions ont été enregistrées dans submission_pred.csv")


NameError: name 'loaded_model' is not defined